<a href="https://colab.research.google.com/github/pallavmarch/Web-Scraping-Projects/blob/main/Imdb_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from rich.console import Console
from rich.table import Table

In [26]:
imdb_charts = {
    "Top Movies": "https://www.imdb.com/chart/top/",
    "Most Popular Movies": "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm",
    "Box Office": "https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht",
    "Top TV Shows": "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250",
    "Most Popular TV Shows": "https://www.imdb.com/chart/tvmeter/?ref_=nv_tvv_mptv",
}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
}

console = Console()
scraped_data = []


def fetch_html(url, max_retries=3):
    """Fetch HTML content with retry mechanism."""
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code == 200:
                return response.text
            console.print(f"[bold red]Failed to retrieve data (Attempt {attempt+1}/{max_retries}): {response.status_code}[/bold red]")
        except requests.exceptions.RequestException as e:
            console.print(f"[bold red]Request failed (Attempt {attempt+1}/{max_retries}): {e}[/bold red]")
        #time.sleep(2)
    return None

def scrape_imdb_chart(chart_name, url):
    """Scrape and display movies/TV shows from IMDb charts."""
    html = fetch_html(url)
    if not html:
        console.print(f"[bold red]Failed to retrieve {chart_name} after multiple attempts.[/bold red]")
        return

    soup = BeautifulSoup(html, "html.parser")
    movies = soup.find_all("li", class_="ipc-metadata-list-summary-item")

    if not movies:
        console.print(f"[bold red]No movies found for {chart_name}. IMDb might have changed its structure.[/bold red]")
        return


    table = Table(title=f"🎬 {chart_name}", header_style="bold magenta")
    table.add_column("Rank", justify="center", style="cyan")
    table.add_column("Title", justify="left", style="yellow")
    table.add_column("Year", justify="center", style="green")
    table.add_column("IMDb Rating", justify="center", style="blue")


    for i, movie in enumerate(movies[:10]):
        title = movie.find("h3", class_="ipc-title__text")
        year = movie.find("span", class_="cli-title-metadata-item")
        rating = movie.find("span", class_="ipc-rating-star--rating")

        title_text = title.text if title else "N/A"
        year_text = year.text if year else "N/A"
        rating_text = rating.text if rating else "N/A"

        table.add_row(f"[bold]{i+1}[/bold]", title_text, year_text, rating_text)
        scraped_data.append({"Chart": chart_name, "Rank": i+1, "Title": title_text, "Year": year_text, "IMDb Rating": rating_text})


    console.print(table)
    #time.sleep(1.5)


for chart_name, url in imdb_charts.items():
    scrape_imdb_chart(chart_name, url)


                                   🎬 Top Movies                                    
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┓
┃ Rank ┃ Title                                                ┃ Year ┃ IMDb Rating ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━┩
│  1   │ 1. The Shawshank Redemption                          │ 1994 │     9.3     │
│  2   │ 2. The Godfather                                     │ 1972 │     9.2     │
│  3   │ 3. The Dark Knight                                   │ 2008 │     9.0     │
│  4   │ 4. The Godfather Part II                             │ 1974 │     9.0     │
│  5   │ 5. 12 Angry Men                                      │ 1957 │     9.0     │
│  6   │ 6. The Lord of the Rings: The Return of the King     │ 2003 │     9.0     │
│  7   │ 7. Schindler's List                                  │ 1993 │     9.0     │
│  8   │ 8. Pulp Fiction                                      │ 1994 │     8.9     │
│  9   │ 9. The Lord of the Rings: The Fellowship of the Ring │ 2001 │     8.9     │
│  10  │ 10. The Good, the Bad and the Ugly                   │ 1966 │     8.8     │
└──────┴──────────────────────────────────────────────────────┴──────┴─────────────┘

                     🎬 Most Popular Movies                     
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┓
┃ Rank ┃ Title                            ┃ Year ┃ IMDb Rating ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━┩
│  1   │ Captain America: Brave New World │ 2025 │     6.1     │
│  2   │ The Gorge                        │ 2025 │     6.8     │
│  3   │ Thunderbolts*                    │ 2025 │     N/A     │
│  4   │ Nosferatu                        │ 2024 │     7.3     │
│  5   │ Bridget Jones: Mad About the Boy │ 2025 │     6.8     │
│  6   │ Anora                            │ 2024 │     7.7     │
│  7   │ The Brutalist                    │ 2024 │     7.9     │
│  8   │ Companion                        │ 2025 │     7.3     │
│  9   │ How to Train Your Dragon         │ 2025 │     N/A     │
│  10  │ The Substance                    │ 2024 │     7.3     │
└──────┴──────────────────────────────────┴──────┴─────────────┘

                           🎬 Box Office                           
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┓
┃ Rank ┃ Title                               ┃ Year ┃ IMDb Rating ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━┩
│  1   │ 1. Captain America: Brave New World │ N/A  │     6.1     │
│  2   │ 2. Paddington in Peru               │ N/A  │     6.8     │
│  3   │ 3. Heart Eyes                       │ N/A  │     6.6     │
│  4   │ 4. Dog Man                          │ N/A  │     6.4     │
│  5   │ 5. Ne Zha 2                         │ N/A  │     8.4     │
│  6   │ 6. Love Hurts                       │ N/A  │     5.4     │
│  7   │ 7. Mufasa: The Lion King            │ N/A  │     6.7     │
│  8   │ 8. One of Them Days                 │ N/A  │     6.8     │
│  9   │ 9. Companion                        │ N/A  │     7.3     │
│  10  │ 10. Becoming Led Zeppelin           │ N/A  │     7.9     │
└──────┴─────────────────────────────────────┴──────┴─────────────┘

                          🎬 Top TV Shows                           
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Rank ┃ Title                           ┃   Year    ┃ IMDb Rating ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│  1   │ 1. Breaking Bad                 │ 2008–2013 │     9.5     │
│  2   │ 2. Planet Earth II              │   2016    │     9.4     │
│  3   │ 3. Planet Earth                 │   2006    │     9.4     │
│  4   │ 4. Band of Brothers             │   2001    │     9.4     │
│  5   │ 5. Chernobyl                    │   2019    │     9.3     │
│  6   │ 6. The Wire                     │ 2002–2008 │     9.3     │
│  7   │ 7. Avatar: The Last Airbender   │ 2005–2008 │     9.3     │
│  8   │ 8. Blue Planet II               │   2017    │     9.3     │
│  9   │ 9. The Sopranos                 │ 1999–2007 │     9.2     │
│  10  │ 10. Cosmos: A Spacetime Odyssey │   2014    │     9.2     │
└──────┴─────────────────────────────────┴───────────┴─────────────┘

                🎬 Most Popular TV Shows                 
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Rank ┃ Title                ┃   Year    ┃ IMDb Rating ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│  1   │ Severance            │   2022–   │     8.7     │
│  2   │ Paradise             │   2025–   │     7.9     │
│  3   │ Apple Cider Vinegar  │   2025    │     7.3     │
│  4   │ Invincible           │   2021–   │     8.6     │
│  5   │ The White Lotus      │   2021–   │     8.0     │
│  6   │ Cobra Kai            │ 2018–2025 │     8.4     │
│  7   │ The Night Agent      │   2023–   │     7.5     │
│  8   │ Yellowstone          │ 2018–2024 │     8.6     │
│  9   │ High Potential       │   2024–   │     7.6     │
│  10  │ Dexter: Original Sin │   2024–   │     8.3     │
└──────┴──────────────────────┴───────────┴─────────────┘